In [1]:
import importlib

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
model_module = importlib.import_module("handyrl.model")
env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")

Loading environment football failed: No module named 'gfootball'


In [3]:
e = env_module.Environment()
e.reset()

In [4]:
obs = e.observation()
obs = torch.from_numpy(obs.reshape(1, 17, 7, 11)).clone().long()
print(f"size: {obs.size()}, type: {obs.dtype}")

size: torch.Size([1, 17, 7, 11]), type: torch.int64


In [5]:
e.obs_list

[[{'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60,
    'step': 0,
    'geese': [[70], [41], [22], [34]],
    'food': [62, 16],
    'index': 0},
   'status': 'ACTIVE'},
  {'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60, 'index': 1},
   'status': 'ACTIVE'},
  {'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60, 'index': 2},
   'status': 'ACTIVE'},
  {'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60, 'index': 3},
   'status': 'ACTIVE'}]]

In [6]:
input_ = torch.randn(2, 17, 7, 11)
print(f"size: {input_.size()}, type: {input_.dtype}")

size: torch.Size([2, 17, 7, 11]), type: torch.float32


In [7]:
net = env_module.GeeseNetIMO(e)

TypeError: unsupported operand type(s) for %: 'tuple' and 'int'

In [ ]:
out = net(obs)
print(f"size: {out.size()}, type: {out.dtype}")

In [ ]:
out

In [ ]:
# pytorch_total_params
params = sum(p.numel() for p in net.parameters())
print(f"{params:,}")

In [ ]:
# pytorch_total_params (trainable)
params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"{params:,}")

In [ ]:
net